In [1]:
DELETE = True
if DELETE:
    !rm -rf *
    print("Delete all...")

Delete all...


In [2]:
!pip install gputil


  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=3afc05e36c284b1984bc0227b782f023dce8ff8d8b26c46ae46ae7c777c88a39
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built gputil


In [3]:
!git clone https://github.com/isl-org/DPT.git

# Download models and weights
#!wget https://github.com/intel-isl/DPT/releases/download/1_0/dpt_hybrid-midas-501f0c75.pt
#!wget https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt
#!wget https://github.com/intel-isl/DPT/releases/download/1_0/dpt_hybrid-ade20k-53898607.pt
!wget https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-ade20k-b12dca68.pt
    
# Import weights
#!mv ./dpt_hybrid-ade20k-53898607.pt ./DPT/weights
!mv ./dpt_large-ade20k-b12dca68.pt ./DPT/weights
#!mv ./dpt_large-midas-2f21e586.pt ./DPT/weights
#!mv ./dpt_hybrid-midas-501f0c75.pt ./DPT/weights

# Pip install required libraries with last releases
!pip install torch
!pip install torchvision
!pip install opencv-python
!pip install timm

Cloning into 'DPT'...
remote: Enumerating objects: 782, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 782 (delta 144), reused 114 (delta 114), pack-reused 606 (from 1)
Receiving objects: 100% (782/782), 458.16 KiB | 5.59 MiB/s, done.
Resolving deltas: 100% (371/371), done.
--2025-06-14 13:50:39--  https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-ade20k-b12dca68.pt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/isl-org/DPT/releases/download/1_0/dpt_large-ade20k-b12dca68.pt [following]
--2025-06-14 13:50:39--  https://github.com/isl-org/DPT/releases/download/1_0/dpt_large-ade20k-b12dca68.pt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-p

In [4]:
!ls

DPT


In [5]:
!cd ../input/dataset-v1/d1/ && ls

informative_signal  prevention_sign  regulation_sign


In [6]:
import os 
def initialize_system():
    # Generate output directory
    if(not(os.path.isdir('/kaggle/working/output'))):
        os.mkdir('/kaggle/working/output')
    
    filename = "/kaggle/working/DPT/run_monodepth.py"
    text = open(filename).read()
    open(filename, "w+").write(text.replace('"output_monodepth"', '"/kaggle/working/DPT/output_monodepth"'))

    filename = "/kaggle/working/DPT/run_segmentation.py"
    text = open(filename).read()
    open(filename, "w+").write(text.replace('"output_semseg"', '"/kaggle/working/DPT/output_semseg"'))
    
    filename = "/kaggle/working/DPT/run_monodepth.py"
    text = open(filename).read()

    #Here goes your files
    open(filename, "w+").write(text.replace('"input"', '"/kaggle/working/DPT/input/"'))

    filename = "/kaggle/working/DPT/run_segmentation.py"
    text = open(filename).read()

    #Here goes your files
    open(filename, "w+").write(text.replace('"input"', '"/kaggle/working/DPT/input/"'))
    
    filename = "/kaggle/working/DPT/run_monodepth.py"
    text = open(filename).read()
    open(filename, "w+").write(text.replace('"weights/', '"/kaggle/working/DPT/weights/'))

    filename = "/kaggle/working/DPT/run_segmentation.py"
    text = open(filename).read()
    open(filename, "w+").write(text.replace('"weights/', '"/kaggle/working/DPT/weights/'))

In [7]:
initialize_system()

In [8]:
# Training parameters
dropout = 0.25
activation = "ReLU"
weight_decay = 0.0
early_stopping = False
learning_rate = 1e-4
batch_size = 4
epochs = 15
optimizer_selected = "Adam"
scheduler_selected = "StepLR"
criterion_selected = "CrossEntropy"
DATASET_PATH = "../input/dataset-v2/d2/"#"../input/dataset-v2/d2/"

In [9]:
with open("DPT/dpt/models.py", "r") as f:
    code = f.read()
# Paso 1: Reemplazar la línea de Dropout fija por una variable
code = code.replace("nn.Dropout(0.1, False)", "dropout_layer")
#code = code.replace("dropout_layer,", "dropout_layer")
if activation == "SiLu":
    code = code.replace(
        "            nn.BatchNorm2d(features),\n            nn.ReLU(True),",
        "            nn.BatchNorm2d(features),\n            nn.SiLU(True),"
    )
elif activation == "GELU":
    code = code.replace(
        "            nn.BatchNorm2d(features),\n            nn.ReLU(True),",
        "            nn.BatchNorm2d(features),\n            nn.GELU(),"
    )
# Paso 3: Asegurar que el constructor tiene los nuevos argumentos
if "dropout_rate" not in code:
    code = code.replace(
        "def __init__(self, num_classes, path=None, **kwargs):",
        "def __init__(self, num_classes, path=None, dropout_rate=0.1, dropout_inplace=False, **kwargs):\n        dropout_layer = nn.Dropout(dropout_rate, inplace=dropout_inplace)\n"
    )

with open("DPT/dpt/models.py", "w") as f:
    f.write(code)

!cat DPT/dpt/models.py

import torch
import torch.nn as nn
import torch.nn.functional as F

from .base_model import BaseModel
from .blocks import (
    FeatureFusionBlock,
    FeatureFusionBlock_custom,
    Interpolate,
    _make_encoder,
    forward_vit,
)


def _make_fusion_block(features, use_bn):
    return FeatureFusionBlock_custom(
        features,
        nn.ReLU(False),
        deconv=False,
        bn=use_bn,
        expand=False,
        align_corners=True,
    )


class DPT(BaseModel):
    def __init__(
        self,
        head,
        features=256,
        backbone="vitb_rn50_384",
        readout="project",
        channels_last=False,
        use_bn=False,
        enable_attention_hooks=False,
    ):

        super(DPT, self).__init__()

        self.channels_last = channels_last

        hooks = {
            "vitb_rn50_384": [0, 1, 8, 11],
            "vitb16_384": [2, 5, 8, 11],
            "vitl16_384": [5, 11, 17, 23],
        }

        # Instantiate backbone and reassemble blocks
    

In [10]:
with open("DPT/dpt/base_model.py", "r") as f:
    code = f.read()

code = code.replace(
    "self.load_state_dict(parameters)",
    """own_state = self.state_dict()
        filtered = {k: v for k, v in parameters.items() if k in own_state and v.shape == own_state[k].shape}
        print(f"Cargando {len(filtered)} de {len(own_state)} parámetros del checkpoint.")
        self.load_state_dict(filtered, strict=False)"""
)


with open("DPT/dpt/base_model.py", "w") as f:
    f.write(code)

In [11]:
# Leer contenido original
with open("DPT/dpt/models.py", "r") as f:
    code = f.read()

# Añadir el nuevo método seguro al final del archivo si no existe
if "def load_partial_weights" not in code:
    code += """

    def load_partial_weights(self, path):
        parameters = torch.load(path, map_location="cpu")
        if "model" in parameters:
            parameters = parameters["model"]

        own_state = self.state_dict()
        filtered = {
            k: v for k, v in parameters.items()
            if k in own_state and v.shape == own_state[k].shape
        }

        print(f"Cargando {len(filtered)} de {len(own_state)} parámetros del checkpoint.")
        self.load_state_dict(filtered, strict=False)
"""



# Guardar cambios
with open("DPT/dpt/models.py", "w") as f:
    f.write(code)

print("Archivo 'models.py' modificado exitosamente.")


!cat DPT/dpt/models.py

Archivo 'models.py' modificado exitosamente.
import torch
import torch.nn as nn
import torch.nn.functional as F

from .base_model import BaseModel
from .blocks import (
    FeatureFusionBlock,
    FeatureFusionBlock_custom,
    Interpolate,
    _make_encoder,
    forward_vit,
)


def _make_fusion_block(features, use_bn):
    return FeatureFusionBlock_custom(
        features,
        nn.ReLU(False),
        deconv=False,
        bn=use_bn,
        expand=False,
        align_corners=True,
    )


class DPT(BaseModel):
    def __init__(
        self,
        head,
        features=256,
        backbone="vitb_rn50_384",
        readout="project",
        channels_last=False,
        use_bn=False,
        enable_attention_hooks=False,
    ):

        super(DPT, self).__init__()

        self.channels_last = channels_last

        hooks = {
            "vitb_rn50_384": [0, 1, 8, 11],
            "vitb16_384": [2, 5, 8, 11],
            "vitl16_384": [5, 11, 17, 23],
        }

        # In

In [12]:
!cat DPT/dpt/base_model.py

import torch


class BaseModel(torch.nn.Module):
    def load(self, path):
        """Load model from file.

        Args:
            path (str): file path
        """
        parameters = torch.load(path, map_location=torch.device("cpu"))

        if "optimizer" in parameters:
            parameters = parameters["model"]

        own_state = self.state_dict()
        filtered = {k: v for k, v in parameters.items() if k in own_state and v.shape == own_state[k].shape}
        print(f"Cargando {len(filtered)} de {len(own_state)} parámetros del checkpoint.")
        self.load_state_dict(filtered, strict=False)


In [13]:
# ===============================
# IMPORTS
# ===============================
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,random_split
from tqdm import tqdm
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import f1_score
from DPT.dpt.models import DPTSegmentationModel

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [14]:
# ===============================
# DATASET CON ALBUMENTATIONS
# ===============================
class RoadSignSegmentationDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform

        subdirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir)
                   if os.path.isdir(os.path.join(root_dir, d))]

        for subdir in subdirs:
            for fname in os.listdir(subdir):
                if fname.endswith("_img.png"):
                    base = fname.replace("_img.png", "")
                    img_path = os.path.join(subdir, f"{base}_img.png")
                    mask_path = os.path.join(subdir, f"{base}_label.png")
                    label_names_path = os.path.join(subdir, f"{base}_label_names.txt")

                    if os.path.exists(mask_path):
                        label_names = []
                        if os.path.exists(label_names_path):
                            with open(label_names_path, 'r') as f:
                                label_names = [line.strip() for line in f.readlines()]

                        self.samples.append({
                            "image": img_path,
                            "mask": mask_path,
                            "labels": label_names
                        })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        image = np.array(Image.open(sample["image"]).convert("RGB"))
        mask = np.array(Image.open(sample["mask"]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented["image"]
            mask = augmented["mask"].long()

        return image, mask, sample["labels"]

In [15]:
# ===============================
# TRANSFORMACIONES
# ===============================
train_transform = A.Compose([
    A.Resize(512, 512),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.GaussNoise(p=0.2),
    A.Affine(
        scale=(0.8, 1.2),
        translate_percent={"x": 0.05, "y": 0.05},
        rotate=(-15, 15),
        p=0.5
    ),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

In [16]:
# ===============================
# MÉTRICAS
# ===============================
def compute_iou(pred, target, num_classes=2):
    ious = []
    for cls in range(num_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds & target_inds).sum()
        union = (pred_inds | target_inds).sum()
        if union == 0:
            ious.append(np.nan)
        else:
            ious.append(intersection / union)
    return np.nanmean(ious)

def pixel_accuracy(pred, target):
    correct = (pred == target).sum()
    total = target.size
    return correct / total


In [17]:
# ===============================
# DATASET
# ===============================
dataset = RoadSignSegmentationDataset(
    root_dir=DATASET_PATH,
    transform=train_transform
)

In [18]:
for image, mask, labels in dataset:
    print(labels)
    break

['_background_', 'prevention']


In [19]:
# ===============================
# CONFIGURACIÓN
# ===============================
NUM_CLASSES = 4
BATCH_SIZE = batch_size
NUM_EPOCHS = epochs
LEARNING_RATE = 1e-5
VAL_SPLIT = 0.2
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===============================
# DATALOADER
# ===============================
val_size = int(VAL_SPLIT * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(len(train_dataset))
print(len(val_dataset))

193
48


In [20]:
# ===============================
# MODELO
# ===============================
model = DPTSegmentationModel(
    num_classes=NUM_CLASSES,
    dropout_rate=dropout,
    backbone="vitl16_384",
    readout="project",
    features=256,
    use_bn=True
)

model.load_partial_weights("DPT/weights/dpt_large-ade20k-b12dca68.pt")
model.to(DEVICE)

Cargando 438 de 442 parámetros del checkpoint.


DPTSegmentationModel(
  (pretrained): Module(
    (model): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in

In [21]:
# ===============================
# CONGELAR BACKBONE
# ===============================
for param in model.pretrained.parameters():
    param.requires_grad = False

In [22]:
print(DEVICE)

cuda


In [23]:
!mkdir ./checkpoints

In [ ]:
import time  # LINEA NUEVA
import pandas as pd  # LINEA NUEVA
import GPUtil  # LINEA NUEVA
from sklearn.metrics import precision_score, recall_score  # LINEA NUEVA

# ===============================
# ENTRENAMIENTO (CONGELADO)
# ===============================
if criterion_selected == "CrossEntropySmoothing":
    criterion = CrossEntropySmoothing(label_smoothing=0.1)
elif criterion_selected == "CrossEntropy":
    criterion = nn.CrossEntropyLoss()

if optimizer_selected == "Adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=weight_decay)
elif optimizer_selected == "AdamW":
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=weight_decay)

if scheduler_selected == "StepLR":
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
elif scheduler_selected == "ReduceLROnPlateau":
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
elif scheduler_selected == "CosineAnnealingLR":
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

metrics = {  # LINEA NUEVA
    "epoch": [],  # LINEA NUEVA
    "train_loss": [], "train_iou": [], "train_precision": [], "train_recall": [], "train_f1": [], "train_acc": [],  # LINEA NUEVA
    "val_loss": [], "val_iou": [], "val_precision": [], "val_recall": [], "val_f1": [], "val_acc": [],  # LINEA NUEVA
    "lr": [], "epoch_time": [], "gpu_mem_mb": []  # LINEA NUEVA
}

patience = 10
patience_counter = 0
best_val_loss = float("inf")

for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch}/{NUM_EPOCHS}")
    start_time = time.time()  # LINEA NUEVA
    model.train()
    running_loss = 0.0
    train_preds, train_targets = [], []  # LINEA NUEVA

    for images, masks, _ in tqdm(train_loader, desc=f"[Train] Epoch {epoch+1}"):
        images, masks = images.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        preds = torch.argmax(outputs, dim=1)
        train_preds.extend(preds.cpu().numpy())  # LINEA NUEVA
        train_targets.extend(masks.cpu().numpy())  # LINEA NUEVA

    # TRAIN METRICS
    train_preds_flat = np.array(train_preds).flatten()  # LINEA NUEVA
    train_targets_flat = np.array(train_targets).flatten()  # LINEA NUEVA
    train_iou = compute_iou(np.array(train_preds), np.array(train_targets))  # LINEA NUEVA
    train_acc = pixel_accuracy(np.array(train_preds), np.array(train_targets))  # LINEA NUEVA
    train_precision = precision_score(train_targets_flat, train_preds_flat, average="macro", zero_division=0)  # LINEA NUEVA
    train_recall = recall_score(train_targets_flat, train_preds_flat, average="macro", zero_division=0)  # LINEA NUEVA
    train_f1 = f1_score(train_targets_flat, train_preds_flat, average="macro", zero_division=0)  # LINEA NUEVA

    # VALIDATION
    model.eval()
    val_loss = 0.0
    val_preds, val_targets = [], []

    with torch.no_grad():
        for images, masks, _ in val_loader:
            images, masks = images.to(DEVICE), masks.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, masks)
            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_targets.extend(masks.cpu().numpy())

    val_preds_flat = np.array(val_preds).flatten()
    val_targets_flat = np.array(val_targets).flatten()
    val_iou = compute_iou(np.array(val_preds), np.array(val_targets))
    val_acc = pixel_accuracy(np.array(val_preds), np.array(val_targets))
    val_precision = precision_score(val_targets_flat, val_preds_flat, average="macro", zero_division=0)
    val_recall = recall_score(val_targets_flat, val_preds_flat, average="macro", zero_division=0)
    val_f1 = f1_score(val_targets_flat, val_preds_flat, average="macro", zero_division=0)

    # GPU + Tiempo
    epoch_time = time.time() - start_time  # LINEA NUEVA
    gpu_mem = GPUtil.getGPUs()[0].memoryUsed if torch.cuda.is_available() else 0  # LINEA NUEVA

    # Log CSV
    metrics["epoch"].append(epoch)  # LINEA NUEVA
    metrics["train_loss"].append(running_loss / len(train_loader))
    metrics["train_iou"].append(train_iou)
    metrics["train_precision"].append(train_precision)
    metrics["train_recall"].append(train_recall)
    metrics["train_f1"].append(train_f1)
    metrics["train_acc"].append(train_acc)
    metrics["val_loss"].append(val_loss / len(val_loader))
    metrics["val_iou"].append(val_iou)
    metrics["val_precision"].append(val_precision)
    metrics["val_recall"].append(val_recall)
    metrics["val_f1"].append(val_f1)
    metrics["val_acc"].append(val_acc)
    metrics["lr"].append(optimizer.param_groups[0]["lr"])  # LINEA NUEVA
    metrics["epoch_time"].append(epoch_time)  # LINEA NUEVA
    metrics["gpu_mem_mb"].append(gpu_mem)  # LINEA NUEVA

    # Guardar CSV
    pd.DataFrame(metrics).to_csv("checkpoints/metrics_log.csv", index=False)  # LINEA NUEVA

    train_loss = metrics["train_loss"][-1]
    val_loss = metrics["val_loss"][-1]
    print(f"[Train] Epoch {epoch+1} | Loss: {train_loss:.4f} | IoU: {train_iou:.4f} | Acc: {train_acc:.4f} | F1: {train_f1:.4f}")
    print(f"[Val]   Epoch {epoch+1} | Loss: {val_loss:.4f} | IoU: {val_iou:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}")

    scheduler.step(val_loss if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau) else None)

    # EARLY STOPPING
    if early_stopping:
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), "checkpoints/best_model.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"⏹️ Early stopping triggered at epoch {epoch+1}")
                break

# Cargar mejor modelo
if early_stopping:
    model.load_state_dict(torch.load("checkpoints/best_model.pt"))
    print("✅ Modelo óptimo cargado desde early stopping.")

# ===============================
# GUARDAR MODELO
# ===============================
os.makedirs("checkpoints", exist_ok=True)
torch.save(model.state_dict(), "checkpoints/dpt_finetuned.pt")
print("Modelo guardado en checkpoints/dpt_finetuned.pt")

# ===============================
# GRÁFICAS
# ===============================
plt.figure(figsize=(12, 8))
plt.plot(metrics["train_loss"], label="Train Loss")
plt.plot(metrics["val_loss"], label="Val Loss")
plt.plot(metrics["train_iou"], label="Train IoU")
plt.plot(metrics["val_iou"], label="Val IoU")
plt.plot(metrics["train_acc"], label="Train Accuracy")
plt.plot(metrics["val_acc"], label="Val Accuracy")
plt.plot(metrics["train_f1"], label="Train F1")
plt.plot(metrics["val_f1"], label="Val F1")
plt.xlabel("Epoch")
plt.ylabel("Valor")
plt.title("Métricas de Entrenamiento y Validación")
plt.grid(True)
plt.legend()
plt.savefig("checkpoints/metrics_plot.png")
plt.show()


Epoch 0/15


[Train] Epoch 1:  10%|█         | 5/49 [00:15<02:15,  3.07s/it]

In [ ]:
import pandas as pd  # Asegurate de tenerlo ya importado
import matplotlib.pyplot as plt
import os

# LINEA NUEVA: Leer el CSV exportado
metrics = pd.read_csv("checkpoints/metrics_log.csv")  # LINEA NUEVA

# LINEA NUEVA: Convertir a listas si es necesario
metrics = {col: metrics[col].tolist() for col in metrics.columns}  # LINEA NUEVA


In [ ]:
import matplotlib.pyplot as plt
import os

# Asegura carpeta de salida
os.makedirs("checkpoints/metric_plots", exist_ok=True)

def plot_metric(train_values, val_values, title, ylabel, filename):
    plt.figure(figsize=(10, 6))
    plt.plot(train_values, label="Train")
    plt.plot(val_values, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(f"checkpoints/metric_plots/{filename}.png")
    plt.show()

# 1. Pérdida (Loss) para detectar overfitting/underfitting
plot_metric(
    metrics["train_loss"],
    metrics["val_loss"],
    "Evolución de la Pérdida",
    "CrossEntropy Loss",
    "loss_comparison"
)

# 2. IoU (Intersección sobre Unión) - Métrica clave para segmentación
plot_metric(
    metrics["train_iou"],
    metrics["val_iou"],
    "Evolución del IoU",
    "IoU",
    "iou_comparison"
)

# 3. Pixel Accuracy - Proporción de píxeles correctamente clasificados
plot_metric(
    metrics["train_acc"],
    metrics["val_acc"],
    "Exactitud por píxel",
    "Pixel Accuracy",
    "accuracy_comparison"
)

# 4. F1 Score - Balance entre precisión y recall
plot_metric(
    metrics["train_f1"],
    metrics["val_f1"],
    "Evolución del F1 Score ",
    "F1 Score",
    "f1_comparison"
)
# 5. Precision
plot_metric(
    metrics["train_precision"],
    metrics["val_precision"],
    "Precisión del modelo",
    "Precision",
    "precision_comparison"
)

# 6. Recall
plot_metric(
    metrics["train_recall"],
    metrics["val_recall"],
    "Recall del modelo",
    "Recall",
    "recall_comparison"
)

# 7. Tiempo por época
plt.figure(figsize=(10, 6))
plt.plot(metrics["epoch_time"], label="Duración por época")
plt.xlabel("Epoch")
plt.ylabel("Segundos")
plt.title("Duración por época")
plt.grid(True)
plt.savefig("checkpoints/metric_plots/epoch_time.png")
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os

# Cargar métricas desde CSV
metrics_path = "checkpoints/metrics_log.csv"
metrics_df = pd.read_csv(metrics_path)

# Crear carpeta de salida
os.makedirs("checkpoints/metric_plots", exist_ok=True)

# Función para graficar y agregar al PDF
def plot_metric_to_pdf(pdf, x, train_values, val_values, title, ylabel):
    plt.figure(figsize=(10, 6))
    plt.plot(x, train_values, label="Train")
    plt.plot(x, val_values, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid(True)
    pdf.savefig()
    plt.close()

# Crear PDF
pdf_path = "checkpoints/metric_plots/metric_report.pdf"
with PdfPages(pdf_path) as pdf:
    x = metrics_df["epoch"]

    plot_metric_to_pdf(pdf, x, metrics_df["train_loss"], metrics_df["val_loss"], "Evolución de la Pérdida", "CrossEntropy Loss")
    plot_metric_to_pdf(pdf, x, metrics_df["train_iou"], metrics_df["val_iou"], "Evolución del IoU", "IoU")
    plot_metric_to_pdf(pdf, x, metrics_df["train_acc"], metrics_df["val_acc"], "Exactitud por píxel", "Pixel Accuracy")
    plot_metric_to_pdf(pdf, x, metrics_df["train_f1"], metrics_df["val_f1"], "Evolución del F1 Score", "F1 Score")
    plot_metric_to_pdf(pdf, x, metrics_df["train_precision"], metrics_df["val_precision"], "Precisión del modelo", "Precision")
    plot_metric_to_pdf(pdf, x, metrics_df["train_recall"], metrics_df["val_recall"], "Recall del modelo", "Recall")
    
    # Plot tiempo por época
    plt.figure(figsize=(10, 6))
    plt.plot(x, metrics_df["epoch_time"], label="Duración por época")
    plt.xlabel("Epoch")
    plt.ylabel("Segundos")
    plt.title("Duración por época")
    plt.grid(True)
    pdf.savefig()
    plt.close()

pdf_path


In [ ]:
!cp checkpoints/metric_plots/metric_report.pdf ./metric_report.pdf
!cp checkpoints/metrics_log.csv ./metrics_log.csv

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

def plot_confusion_matrix(model, dataloader, device, num_classes=2, normalize=True, title="Matriz de Confusión"):
    all_preds = []
    all_targets = []

    model.eval()
    with torch.no_grad():
        for images, masks, _ in dataloader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy().flatten())
            all_targets.extend(masks.cpu().numpy().flatten())

    cm = confusion_matrix(all_targets, all_preds, labels=list(range(num_classes)), normalize='true' if normalize else None)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[f"Clase {i}" for i in range(num_classes)])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(title)
    plt.grid(False)
    plt.savefig("checkpoints/confusion_matrix.png")
    plt.show()

plot_confusion_matrix(model, val_loader, DEVICE, num_classes=2)


In [ ]:
# ===============================
# EVALUADOR DE MODELO
# ===============================
def evaluate_model(model_path, dataloader, device, num_classes=2, pretrained=True):
    if pretrained:
        model = DPTSegmentationModel(
            num_classes=num_classes,
            path=model_path,
            backbone="vitl16_384",
        )
    else:
        model = DPTSegmentationModel(
            num_classes=num_classes,
            path=None,
            backbone="vitl16_384",
        )
        model.load_partial_weights(model_path)

    model.to(device)
    model.eval()

    iou_scores = []
    pixel_accuracies = []
    f1_scores = []

    with torch.no_grad():
        for images, masks, labels in tqdm(dataloader, desc=f"Evaluando {model_path}"):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            for p, t in zip(preds, masks):
                p_np = p.cpu().numpy()
                t_np = t.cpu().numpy()
                iou_scores.append(compute_iou(p_np, t_np, num_classes))
                pixel_accuracies.append(pixel_accuracy(p_np, t_np))
                f1_scores.append(f1_score(t_np.flatten(), p_np.flatten(), average="macro"))

    return {
        "IoU": np.nanmean(iou_scores),
        "Pixel Accuracy": np.mean(pixel_accuracies),
        "F1 Score": np.mean(f1_scores)
    }

In [ ]:
eval_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

eval_dataset = RoadSignSegmentationDataset(DATASET_PATH, transform=eval_transform)
eval_loader = DataLoader(eval_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Evaluación
print("Evaluando modelo preentrenado...")
results_pre = evaluate_model("DPT/weights/dpt_large-ade20k-b12dca68.pt", eval_loader, DEVICE, NUM_CLASSES, pretrained=True)
print("Modelo Preentrenado:", results_pre)

print("Evaluando modelo fine-tuned...")
results_fine = evaluate_model("checkpoints/dpt_finetuned.pt", eval_loader, DEVICE, NUM_CLASSES, pretrained=False)
print("Modelo Fine-Tuned:", results_fine)


In [ ]:
# ================================================
# INTERACTIVO: ELECCIÓN DE MODELO + SUBIDA DE IMAGEN
# ================================================
from IPython.display import display
from ipywidgets import Dropdown, FileUpload
import io
from PIL import Image
import torch
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt

# -------- Dropdown para elegir modelo ----------
model_selector = Dropdown(
    options=[('Fine-tuned (model)', 'model'), ('Preentrenado (model2)', 'model2')],
    value='model',
    description='Modelo:',
)
display(model_selector)

# -------- Subida de imagen ----------
uploader = FileUpload(accept='image/*', multiple=False)
display(uploader)


In [ ]:
# ================================================
# PROCESAMIENTO Y PREDICCIÓN - VERSIÓN ROBUSTA
# ================================================

if uploader.value:
    # Detectar tipo de estructura
    if isinstance(uploader.value, dict):
        uploaded_file = list(uploader.value.values())[0]
    elif isinstance(uploader.value, tuple):
        uploaded_file = uploader.value[0]
    else:
        raise ValueError("Formato de archivo no reconocido")

    # Obtener imagen
    image_bytes = uploaded_file['content']
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    orig_image = image.copy()

    # Transformación
    transform = T.Compose([
        T.Resize((512, 512)),
        T.ToTensor(),
        T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ])
    input_tensor = transform(image).unsqueeze(0).to(DEVICE)

    # Selección de modelo
    selected_model = model if model_selector.value == 'model' else model2
    selected_model.to(DEVICE)
    selected_model.eval()

    with torch.no_grad():
        output = selected_model(input_tensor)
        pred_mask = torch.argmax(output, dim=1).squeeze().cpu().numpy()

    # Visualización
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(orig_image)
    plt.title("Imagen Original")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(pred_mask, cmap="gray")
    plt.title("Máscara Predicha")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

else:
    print("⬆️ Subí una imagen para visualizar la predicción.")


In [ ]:
# Descongelar todo
for param in model.parameters():
    param.requires_grad = True

# O: solo descongelar encoder
for param in model.pretrained.parameters():
    param.requires_grad = True

# Nuevo optimizador con lr más bajo
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


In [ ]:
!mv /kaggle/working/checkpoints/dpt_finetuned.pt /kaggle/working/output

In [ ]:
!ls -la /kaggle/working/output/dpt_finetuned.pt
!date